In [10]:
import face_recognition
import cv2
import os
import pickle
import time
import numpy as np
import imutils
print(cv2.__version__)

4.5.3


In [11]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/face_detection_model/"
encodings_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_encodings.pickle"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_recognizer.pickle"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_label_encoder.pickle"


In [12]:
# Load face detector model
print("[INFO] loading face detector...")
protoPath = os.path.join(detector_main_folder, "deploy.prototxt")
modelPath = os.path.join(detector_main_folder,"res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("[INFO] face detector loaded.")
# load the actual face recognition model along with the label encoder
print("[INFO] loading trained face recognizer model...")
recognizer = pickle.loads(open(recognizer_save_path, "rb").read())
le = pickle.loads(open(label_encoder_save_path, "rb").read())

[INFO] loading face detector...
[INFO] face detector loaded.
[INFO] loading trained face recognizer model...


In [14]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
cam = cv2.VideoCapture(0)
print('Is the IP camera turned on: {}'.format(cam.isOpened()))

print("CV_CAP_PROP_FRAME_WIDTH: '{}'".format(cam.get(cv2.CAP_PROP_FRAME_WIDTH))) 

print("CV_CAP_PROP_FRAME_HEIGHT : '{}'".format(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))) 

print("CAP_PROP_FPS : '{}'".format(cam.get(cv2.CAP_PROP_FPS))) 

print("CAP_PROP_POS_MSEC : '{}'".format(cam.get(cv2.CAP_PROP_POS_MSEC))) 

print("CAP_PROP_FRAME_COUNT  : '{}'".format(cam.get(cv2.CAP_PROP_FRAME_COUNT))) 

print("CAP_PROP_BRIGHTNESS : '{}'".format(cam.get(cv2.CAP_PROP_BRIGHTNESS))) 

print("CAP_PROP_CONTRAST : '{}'".format(cam.get(cv2.CAP_PROP_CONTRAST))) 

print("CAP_PROP_SATURATION : '{}'".format(cam.get(cv2.CAP_PROP_SATURATION))) 

print("CAP_PROP_HUE : '{}'".format(cam.get(cv2.CAP_PROP_HUE))) 

print("CAP_PROP_GAIN  : '{}'".format(cam.get(cv2.CAP_PROP_GAIN))) 

print("CAP_PROP_CONVERT_RGB : '{}'".format(cam.get(cv2.CAP_PROP_CONVERT_RGB)))

while(True):
    
    _,frame=cam.read()
    time_1 = time.time()
    image = imutils.resize(frame, width=600)
    (h, w) = image.shape[:2]
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections
        if confidence > DETECT_CONF_THRESH:
            # compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Need to convert because the face_recognition.face_encoding() doesn't use coordinate system
            box = [(startY, endX, endY, startX)]
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Compute encoding using face_recognition library
            encodings = np.array(face_recognition.face_encodings(rgb, box))

            # perform classification to recognize the face
            preds = recognizer.predict_proba(encodings)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # draw the bounding box of the face along with the associated probability
            text_1 = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
#             if( true_name == name):
#                 color = (0, 255, 0)
#             else:
#                 color = (0, 0, 255)
            cv2.putText(image, text_1, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,0), 2)
    # show the output image
    cv2.imshow("Image", image)
    time_2 = time.time()
    print(f"FPS: {1/(time_2 - time_1):.2f}")
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

Is the IP camera turned on: True
CV_CAP_PROP_FRAME_WIDTH: '640.0'
CV_CAP_PROP_FRAME_HEIGHT : '480.0'
CAP_PROP_FPS : '30.0'
CAP_PROP_POS_MSEC : '0.0'
CAP_PROP_FRAME_COUNT  : '-1.0'
CAP_PROP_BRIGHTNESS : '128.0'
CAP_PROP_CONTRAST : '32.0'
CAP_PROP_SATURATION : '64.0'
CAP_PROP_HUE : '0.0'
CAP_PROP_GAIN  : '4.0'
CAP_PROP_CONVERT_RGB : '1.0'
FPS: 0.89
FPS: 1.21
FPS: 1.29
FPS: 1.40
FPS: 1.45
FPS: 16.56
FPS: 17.01
FPS: 17.10
FPS: 16.26
FPS: 17.11
FPS: 19.44
FPS: 15.56
FPS: 16.68
FPS: 17.82
FPS: 16.91
FPS: 18.69
FPS: 14.59
FPS: 25.12
FPS: 25.80
FPS: 26.95
FPS: 7.83
FPS: 9.51
FPS: 10.07
FPS: 10.37
FPS: 26.80
FPS: 8.60
FPS: 6.66
FPS: 7.65
FPS: 27.18
FPS: 8.71
FPS: 8.80
FPS: 18.48
FPS: 17.95
FPS: 16.99
FPS: 14.06
FPS: 16.73
FPS: 18.24
FPS: 16.65
FPS: 15.64
FPS: 19.05
FPS: 25.40
FPS: 27.71
FPS: 7.97
FPS: 8.60
FPS: 13.01
FPS: 15.39
FPS: 8.52
FPS: 8.74
FPS: 24.93
FPS: 13.24
FPS: 11.81
FPS: 11.96
FPS: 16.96
FPS: 17.11
FPS: 9.37
FPS: 9.55
FPS: 10.08
FPS: 8.75
FPS: 23.67
FPS: 22.57
FPS: 20.69
FPS: 8.92